# Программирование на языке Python. Уровень 1.Основы языка Python

## Модуль 9. Элементы функционального программирования, практика

Из материалов курса мы помним, что функция:
- может быть сохранена в переменной или структуре данных; 
- может быть передана в другую функцию как аргумент; 
- может быть возвращена из функции как результат; 
- может быть создана во время выполнения программы; 
- не должна зависеть от именования. 

Рассмотрим наиболее частые случаи специфичного применения функций в python.

- функции как переменные:
    - функция - конструктор;
    - что такое замыкание;
    - что такое каррирование;
    - что такое декораторы.



### Функция - конструктор

Функция может возвращать не только какие-либо значения, но и другую функцию. Это часто встречается, например, в библиотеках машинного обучения ```sklearn``` и ```keras```.

Рассмотрим пример функции, которая создает линейную функцию:

In [ ]:
def linear_builder(k, b): 
    def helper(x): 
        return k * x + b 
    return helper

linear = linear_builder(1, 2)

print(linear(-2))
print(linear(-1))
print(linear(0))
print(linear(1))
print(linear(2))

### Функция - замыкание

Замыкание (closure) — функция, которая находится внутри другой функции и ссылается на переменные, объявленные в теле внешней функции (свободные переменные).

Внутренняя функция создается каждый раз во время выполнения внешней. Каждый раз при вызове внешней функции происходит создание нового экземпляра внутренней функции, с новыми ссылками на переменные внешней функции.

Ссылки на переменные внешней функции действительны внутри вложенной функции до тех пор, пока работает вложенная функция, даже если внешняя функция закончила работу, и переменные вышли из области видимости.

Например, функция ```helper()``` внутри ```linear_builder()``` - это замыкание. Переменная ```b_```, заданная в функции ```linear_builder()``` будет использоваться внутри функции ```helper()``` пока существуют функции, порожденные функцией ```linear_builder()```, и у каждой такой функции будут свои значения ```k``` и ```b```.

In [ ]:
def linear_builder(k, b=None): 
    b_ = b if b is not None else 100
    def helper(x): 
        print('k, b_:', k, b_)
        return k * x + b_ 
    return helper

linear_5 = linear_builder(5)
print(linear_5(0))

linear_1_8 = linear_builder(1, 8)
print(linear_1_8(0))

print(linear_5(1))
print(linear_5(100500))

print(linear_1_8(1))
print(linear_1_8(100500))

### Каррирование

Каррирование - это перевод функций, принимающих набор параметров ```f(x_1, x_2, .., x_n)``` в набор последовательно вызываемых функций ```f(x_1)(x_2)...(x_n)```. Этот прием бывает полезен при возможном частичном использовании функций.

In [ ]:
def f_add(x_1, x_2, x_3):
    return x_1 + x_2 + x_3

def f_add_curry(x_1):
    def helper_1(x_2):
        def helper_2(x_3):
            return x_1 + x_2 + x_3
        return helper_2
    return helper_1


print(f_add(2,2,0))
print(f_add_curry(2)(2)(0))

fn_2_plus_2_plus_smth = f_add_curry(2)(2)
fn_2_plus_2_plus_smth(0)

### Декораторы

Декораторы - это "обертки" для выполнения рутинных операций при вызове функций (например, препроцессинга, постпроцессинга или контроля входных данных), для них существует специальная синтаксическая конструкция.

Предположим, что в нашем проекте нам необходимо перед вызовом и после вызова ряда функций печатать информационные сообщения. Добавлять код, печатающий эти сообщения в каждую функцию - это нарушение правила DRY (Don't Repeat Yourself). Можно прибегнуть к замыканиям и написать вот такую функцию:



In [ ]:
def info_print(fn):
    def fn_to_ret( *args, **kwargs ):
        print('Before the call ...')
        fn( *args, **kwargs )
        print('After the call ...')
    return fn_to_ret

def fn_test1():
    print('Hi from fn_test1')
    
info_print(fn_test1)() # так себе конструкция

print('---')

fn_test1_pre_post = info_print(fn_test1)

fn_test1_pre_post() # тоже не очень

print('---')

@info_print
def fn_test2():
    print('Hi from fn_test2')
    
fn_test2()

### Парадигма map/reduce

"Разбить, обработать данные, собрать". Несмотря на отмирание функционального программирования в чистом виде, эта парадигма сегодня снова стала актуальной, особенно в области анализа больших данных. Благодаря ей можно организовывать распределенные вычисления, например, в задачах машинного обучения. Это делается более продвинутыми средствами (не базовыми средствами python, а, к примеру, Apache Spark). Тем не менее, реализация map/reduce в python позволяет распараллелить вычисления на несколько потоков в рамках одного компьютера (сервера).

Функция ```map(func, *iterables)``` применяет функцию ```func()``` ко всем элементам ```iterables``` и возвращает итерируемый объект, который состоит из преобразованных элементов. Этот объект можно "прокрутить" в цикле ```for ... in ...``` или превратить в список функцией ```list()```.

In [ ]:
dict_ = { 'James Brown': 'musician', 
          'Luke Skywalker': 'character', 
          'John Lennon': 'musician',
          'Duke Ellington': 'musician',
          'Cinderella': 'character'
         }

mapped = map(lambda name: f"{name}, {dict_[name]}", dict_)

print(type(mapped))

for x in mapped:
    print(x)
    
print(list(mapped)) # пустой список! 
# чтобы работать полученными данными как со списком, сразу преобразуйте map в список и запишите его в переменную

Функция ```filter(func, *iterables)``` формирует итерируемый объект из тех позиций ```iterables```, для которых функция ```func()``` вернула ```True```.

In [ ]:
filtered = filter(lambda name: dict_[name]=='musician', dict_)
print(list(filtered))
print(list(filtered)) # то же самое происходит с объектом filter - он "исчерпывается"

Функция ```reduce(func, iterable[, initial])``` реализует кумулятивные вычисления на базе функции ```func(x_1, x_2)```. Эта функция принимает два значения: первое - уже вычисленное функцией ```reduce()```, второе - текущий элемент из ```iterable```. В начале цикла ```reduce()``` берет либо значение ```initial```, а если он не указан - первый элемент из ```iterable```.

In [ ]:
from functools import reduce

def collect_musicians(list_, name_role):
    name, role = name_role
    if role=='musician':
        list_.append(name)
    return list_
    

musicians = reduce(collect_musicians, dict_.items(), [])
print(musicians)
print(type(musicians))

#### ПРАКТИКА

Выполните следущие задачи в "клетках" данной тетради.

1. Простейшие задачи. Используйте переменные, импорт модуля math, циклы и функции.

In [ ]:
# выведите число "Пи"

# посчитайте длину окружности и площадь круга радиусом 5 см, выведите ответ в дружественной форме

# посчитайте площадь квадрата, вписанного в этот круг, выведите ответ в дружественной форме

# создайте список действительных чисел от 5 до 7 с шагом 0.05

# посчитайте длины окружностей, площади кругов и площади квадратов, которые можно вписать в эти круги
# для всех элементов этого списка

# выведите ответ в виде таблицы


2. Напишите программу, которая преобразует в сантиметры значения в футах и дюймах, которые пользователь задает в виде строки &lt;ft&gt;'&lt;in&gt;\" (кол-во футов, символ одинарных кавычек, кол-во дюймов, символ двойных кавычек), например, 5\'8\", 3\'2\"

In [ ]:
# ваш код здесь


3. Постройте горизонтальную гистограмму по значениям из заданного списка длиной 10. Для заполнения используйте символ "O"

In [ ]:
list_ = [0,1,1,2,7,4,3,2,1,0]

# ваш код здесь

4. ...а теперь постройте вертикальную гистограмму для этого списка

In [ ]:
# ваш код здесь

5. Получите список всех хештегов, упомянутых в тексте.

In [ ]:
text = """
Этот #пост создан для того, чтобы его #лайк'али. 
Посмотрите, какая красивая фотография #котик'а. 
Какой он #Мягкий_и_Пушистый!
Пройдите #опрос, он у нас посвящен #кот'ам.
Всем #чао! ###
"""

re_tag = re.compile(r"\#[\w]+")
# ваш код здесь
